# Lecture 3: Recursion and Induction

In this lecture, our plan is to investigate the following concepts.

- We will start by  looking afresh at recursive functions.
  - Termination, Recursion depth and Number of recursive calls.
- Tail recursion.
- Making functions tail recursive.
- Fun (?) with recursion.


## Recursive Functions.

A function $f$ is defined recursively if the body of the definition refers back to $f$. The easiest example is that of the `factorial` function, which we have already encountered before.


In [1]:
def factorial(x: Int): Int = {
    if (x == 0) {
       1 // This is called the base case
    } else { 
       x * factorial(x-1) // The recursion is here.   
    }
}

defined function factorial

In [2]:
val x1 = factorial(5)
val x2 = factorial(10)

x1: Int = 120
x2: Int = 3628800

Factorial is recursive since in its body we are refering 
back to the function itself. This seems like a circular definition,
and certainly prone to lots of abuse. For example, take a look at the `myCrazyMeaninglessVeryBadFunction` below. What is wrong with this function?

In [3]:
def myCrazyMeaninglessVeryBadFunction(x: Int): Int = {
    if (x == 0 ){
        1 // The base case
    } else { 
        x * myCrazyMeaninglessVeryBadFunction(x) // The recursion
    }
}

defined function myCrazyMeaninglessVeryBadFunction

The answer is simple: `factorial` _seems to_ terminate to provide an answer for all inputs, whereas `myCrazy...Function` simply does not terminate for any input. 

We will first consider simple recursive definitions that have the following form:

$$ f(x) = \left\{ \begin{array}{ll}
    \text{constant} & \text{if (base case condition holds)} \\
    \text{.. expression involving f .. } & \text{otherwise} \\
    \end{array}\right.$$

Therefore, for any input $x_0$, we expect to observe a sequence of recursive calls: 

$$ f(x_0) \rightsquigarrow f(x_1) \rightsquigarrow \cdots \rightsquigarrow f(x_N) \rightsquigarrow \cdots $$

We want that for every $x_0$ (starting value), the sequence obtained always reaches the base case condition and exits.

__Definition (Termination)__ A recursive definition of a function $f(x)$ is terminating if and only if the sequence of recursive calls for any value of $x$ eventually hits the base case of the recursion.

We already see that `myCrazy...Function` is not terminating and the reason is obvious: `myCrazy...Function(x)` calls back `myCrazy...Function(x)` thus going into an infinite loop.

On the other hand, `factorial(x)` calls `factorial(x-1)`, which in turn calls `factorial(x-2)`, and so on. It is easy to see that for any $ x \geq 0$, this process of unwinding the definitions will eventually get us to the base case `factorial(0)` and thus terminate. 

Therefore, is the factorial function terminating? Unfortunately, not as defined. This is where you have to be careful. It seems to terminate for inputs such as $2$.

$$\text{factorial(2)} \rightsquigarrow \text{factorial(1)} \rightsquigarrow \text{factorial(0)} \,.$$

But what about `factorial(-2)`? It exposes the issue.

$$\text{factorial(-2)} \rightsquigarrow \text{factorial(-3)} \rightsquigarrow \text{factorial(-4)} \rightsquigarrow \cdots \,.$$

The reader at this point may argue that `factorial(n)` was defined only for positive integers $n$. However, this is not reflected in the type of the scala function: `factorial( x: Int): Int` which promises to return an integer whenever the input is an integer (that includes both positive and negative numbers).

## Preconditions

A precondition is a constraint that restricts what inputs can be used to call a function. 

For instance, the `factorial` function has the precondition that its input must be non negative. In scala we can use the `require` keyword to specify a precondition.


In [4]:
def factorialWithPreconds(x: Int): Int = {
    require(x >= 0)
    if (x == 0) {
       1 // This is called the base case
    } else { 
       x * factorialWithPreconds(x-1) // The recursion is here.   
    }
}

defined function factorialWithPreconds

In [5]:
val y = factorialWithPreconds(2)

y: Int = 2

In [6]:
val z = factorialWithPreconds(-2)

: 

## Preconditions versus Default Values

Preconditions are very useful in software engineering practice. They expose the designer's expectations on what the inputs to a function should look like so that the execution can proceed without bugs. It is an important habit to try and write preconditions whenever appropriate.

Another approach is to simply return a default value (say `-1` or `0`) when evaluating a function. The advantage is that it allows any input to execute without throwing an exception. However, the key disadvantage is that it imposes a requirement that the result of the function always be checked by the caller and the default values handled appropriately.  Failing such a check often leads to _silent failures_ or failures that are see far away from the true cause. 


## Termination and Ranking Functions

The question of whether a recursive definition terminates is very important and at the same time a very hard problem. The main tool here is to show that the sequence of recursive calls from an input x _makes progress_ towards the base case condition. 

Consider the function `factorialWithPrecond(x: Int)` with input $x$ that satisfies the precondition $x \geq 0$. 

- The base case of this call is $x = 0$.
- Each call to `factorialWithPrecond(x)` with $x > 0$, results in the recursive call to `factorialWithPrecond(x - 1)`.

Thus, we can combine the two statements to conclude that any call to `factorialWithPrecond(x)` will terminate in $(1 + x)$ steps provided $x \geq 0$.

Let us take another slightly more complex definition.


In [7]:
def fibonacci(n: Int): Int = {
    require ( n >= 0 )
    if (n <= 1) { 1 }
    else {
        fibonacci(n-1) + fibonacci(n-2)
    }
}

defined function fibonacci

`fibonacci` computes the $n^{th}$ Fibonacci number for $n \geq 0$. We have added the precondition $n \geq 0$ to enure that it is never callable on negative inputs and such. Is it terminating?

A big difference between `factorial` and `fibonacci` is that while the former recursion involves just one call back to itself, `fibonacci` calls itself back twice. 

However, is it terminating?

Yes it is and the same principles we used to understand factorial function can help us here. Each call to `fibonacci(n)` calls back `fibonacci(n-1)` and `fibonacci(n-2)`. The termination can be easily established through induction.
- Base Case: `fibonacci(0)` terminates.
- Induction Step: If `fibonacci(k)` terminates for all $ 0 \leq k \leq n-1$ and $n \geq 1$, then so does `fibonacci(n)`.
  - Case 1: $ n = 1$. The base case applies and it terminates
  - Case 2: $ n \geq 2$. 
    - We can apply induction to see that both recursive calls `fibonacci(n-1)` and `fibonacci(n-2)` terminate. 
    - To do so understand that the inductive hypothesis applies to $0 \leq k \leq n-1$. 
    - We verify that  $0 \leq n-1 \leq n-1$ and $0 \leq n-2 \leq n-1$.
- Therefore, we can conclude that `fibonacci(n)` terminates.



In [8]:
// Here are two recursions that look different.
// Are they terminating?
def isPowerOfTwo(x: Int): Boolean = {
    // Check if x is a power of two
    require( x >= 0)
    if (x == 0) { false }
    else if (x == 1) { true }
    else if (x % 2 == 1) { false }
    else {
        isPowerOfTwo(x / 2)
    }
}

def recurseToPowerOfTwo(x: Int): Int = {
    require( x >= 0)
    if (isPowerOfTwo(x)) { x }
    else {
        recurseToPowerOfTwo(x + 1)
    }
}

defined function isPowerOfTwo
defined function recurseToPowerOfTwo

In [9]:
val z = isPowerOfTwo(15)
val w = isPowerOfTwo(32)

z: Boolean = false
w: Boolean = true

In [10]:
val e1  = recurseToPowerOfTwo(35)

e1: Int = 64

In [11]:
val e2  = recurseToPowerOfTwo(135)

e2: Int = 256

In [12]:
// Here is a famous recursion due to Manna, Pnueli and McCarthy. 
// Can you guess what it does?

def M(x: Int): Int = {
    if (x > 100) { x - 10 }
    else { M(M( x+11 ))}
}

defined function M

In [13]:
val x1 = M(-20)

x1: Int = 91

In [14]:
val x2 = M(145)

x2: Int = 135

In [15]:
val x3 = M(55)

x3: Int = 91

In [16]:
// Termination proof of this function is open.
// See Collatz problem: which asks if this recursive
// function below terminates on all inputs.

def collatz(x: Int): Int = {
    require (x >= 0)
    if (x == 1) { 1 }
    else if (x % 2 == 1) { collatz (3 * x + 1 )}
    else { collatz (x / 2 )}
}

defined function collatz

In [17]:
val v1 = collatz(51)
val v2 = collatz(79)
val v3 = collatz(352)

v1: Int = 1
v2: Int = 1
v3: Int = 1

## Recursion Tree, (Stack) Depth and Number of Calls

You must be familiar from your computer systems classes as to how function calls are executed on a computer. 
- The system maintains a call stack with an `activation record` for each function call. 
- When a function is called, a new activation record is created for the called function that includes the return address (where in the program to return to when the call returns), the values of function call parameters and local variables to the function.
- When a function returns, the control passes back to its caller at the return address stored in the stack.

### Example

Consider the following code snippet:


In [18]:
def f(x: Int): Int = {
    x * 5
}

def g(y: Int): Int = {
     val tmp = f(y)
     tmp * 10
}

def h(z: Int): Int = {
    val tmp2 = g(z)
    tmp2 + 5
}

h(15)
// stuff that follows h(15)

defined function f
defined function g
defined function h
res17_3: Int = 755

The call to `h(15)` causes an activation record for `h` to be created. We will not really go into the details of how the Java Virtual Machine (JVM) does activation records.

The activation record for the call to h looks somewhat like a table with the following information:

~~~
Activation record for h
Return Address: Line 16.
z: 15
tmp2: ....
~~~

Now `h` executes with `z: 15` and it issues a call to `g` with `z` as the argument. 
Therefore, a new activation record is created and placed on top of the stack.

~~~
Activation record for g.
Return Address: line 12.
y: 15
tmp: ...

Activation record for h
Return Address: Line 16.
z: 15
tmp2: ...
~~~

Finally, `f` is called and a new activation record for `f` is created and placed on top of the stack.

~~~
Activation record for f.
Return Address: line 7
x: 15

Activation record for g.
Return Address: line 12.
y: 15
tmp: ...

Activation record for h
return Address: Line 16.
z: 15
tmp2: ...
~~~

When `f` finished executing, it returns `75` that gets placed in the val `tmp` for `g`. The activation record for `f` is taken out of the stack and the modified record for `g` looks like this.

~~~

Activation record for g.
Return Address: line 12.
y: 15
tmp: 75 

Activation record for h
return Address: Line 16.
z: 15
tmp2: ...
~~~

Similarly, `g` finishes its execution and returns 750 back to its caller.

~~~
Activation record for h
return Address: Line 16.
z: 15
tmp2: 750
~~~

As you can see the stack grows with a function call when a new activation record is added and shrinks when a function returns.



### Recursive Calls

Thus, recursive calls are implemented much like any other function call. However, because these functions call themselves, the stack grows as recursive calls are made. We are interested in two aspects of the resource consumption:

- __Depth of Recursion:__ how many activation records can reside in the stack at any point during the execution of the recursive function, in the worst case?
- __Number of Recursive Calls:__ How many calls are made to the recursive function in total?

<todo illustrate this with diagrams in class>

To facilitate this analysis, we can view the execution of a recursion as a tree wherein the root of the tree is the very first recursive call. For each node, the children are just the recursive calls made by that node. Leaves of the tree correspond to calls that fall into the base cases.

<todo illustrate with a diagram>

The depth of the tree is therefore the depth of the recursion. The number of recursive calls is the number of nodes in the tree.


### Factorial Function

Let us draw this tree for `factorial(5)`, having fixed the issue for factorial by adding `require(x >= 0)`.



In [19]:
def factorial(x : Int): Int = {
    require( x >= 0) // Fixed the issue!
    if (x == 0 ) 1 else {x * factorial(x-1)}
}

defined function factorial

Here is the tree

<img src="http://www.cs.colorado.edu/~srirams/courses/csci3155-fall2018/pictures/factorial-5-tree.png" width="10%">

It is easy to see that the depth equals $6$ and the number of calls is also $6$.

For general $n$, `factorial(n)` has stack depth $n+1$ and the number
of calls is the same as the stack depth.

### Fibonacci Function

Let us draw the tree for `fibonacci(4)`. 

<img src="http://www.cs.colorado.edu/~srirams/courses/csci3155-fall2018/pictures/fibonacci-tree-4.png" width="75%">
    
   
The stack depth is 4, but the number of calls is 9.
    
In general, the depth of `fibonacci(n)` is $n$. However, the number
of calls is a different story.

| n | Number of calls in `fibonacci(n)` |
|---|-----------------------------------|
| 0 |  1 | 
| 1 |  1 | 
| 2 |  3 | 
| 3 |  5 | 
| 4 |  9 | 
| 5 |  15 | 

What is the pattern here?  Assuming $ n \geq 2$, 

`# calls to fibonacci(n) = 1 + # calls to fibonacci(n-1) + # calls to fibonacci(n-2) `
    
Thus, the recurrence for number of calls is given by

$$C(n) = \left\{ \begin{array}{rl}
1 & n \leq 1 \\
1 + C(n-1) + C(n-2) & \mbox{otherwise} \\
\end{array}\right.$$

These are called Leonardo numbers: https://oeis.org/A001595

Unfortunately, the growth of the number of recursive calls is exponential in $n$. Thus, to compute `fibonacci(40)` requires us to make more than a billion calls. 

## Tail Calls

There is a very special case where the activation records do not have to grow upon successive function calls. These are called *tail calls*. Let us illustrate them with an example. 

**Example 1:** We already saw this example from above and traced out how the stack grows when the successive calls are made.

In [20]:
def f(x: Int): Int = {
    x * 5
}

def g(y: Int): Int = {
     val tmp = f(y)
     tmp * 10
}

def h(z: Int): Int = {
    val tmp2 = g(z)
    tmp2 + 5
}

h(15)
// stuff that follows h(15)

defined function f
defined function g
defined function h
res19_3: Int = 755

**Example 2:** Consider a different example below and carefully compare this code to that above.

In [21]:
def f1(x: Int): Int = {
    x * 8
}

def g1(y: Int): Int = {
     val tmp = 12 * y
     f1(tmp)
}

def h1(z: Int): Int = {
    val tmp2 = 14 + z
    g1(tmp2)
}

h1(15)
// stuff that follows h(15)

defined function f1
defined function g1
defined function h1
res20_3: Int = 2784

Obviously, the functions are doing something totally different. But let us point out an important difference between the function call `g(tmp2)` at line 12 of example 2 and the corresponding call `val tmp2 = g(z)`  from line 11 of example 1. 

A key difference is that the result of the call `g(tmp2)` in example 2 is returned back to the callee without any further computations whereas in example 1, the result is actually processed further by adding 5 and then returned.

**Definition (Tail Call)** A function call `f(...)` is said to be a tail call if (a) no further computation is performed when the call to `f` returns back and (b) the result is passed back to the caller (without any modifications).

For example all function calls in example 2 are tail calls whereas the calls in example 1 are not tail calls. 

Tail calls are important because they allow the system to perform an important optimization called *tail call optimization*.

### Tail Call Optimization

Let us see how this works with example 2. Consider the activation stack when `g1` is called inside function `h1`.

~~~
Activation Record for h1
Return Address: line 16
z: 15
tmp2: 29
~~~

Normally, we will now call `g1(29)` and therefore a new activation record is added.

~~~
Activation Record for g1
Return Address: line 13
y: 29
tmp: ... 

Activation Record for h1
Return Address: line 16
z: 15
tmp2: 29
~~~

The key question is whether we need this? What happens when `g1` returns? Because it was called as a tail call, the value returned back to `h1` is just sent back to the caller.
Tail call optimization is a very simple trick. Rather than keep the activation record for `h1` around, it simply **replaces** the activation record for `h1` as follows:


~~~
Activation Record for g1 (TAIL CALL OPTIMIZED)
Return Address: line 16
y: 29
tmp: ... 
~~~

There is a very key change in terms of the return address. Rather than return back from `g1` to `h1` and from `h1` to its caller, we will bypass the *middle man* and directly send our result to whosoever was waiting for `h1`.

As a result of tail call optimization, we conclude that **tail calls need not cause the stack size to increase**.

## Tail Recursion

We will now look closer into tail recursion, which we briefly introduced earlier.  Let us look at two examples of recursions.

In [22]:
def recursionA(n: Int): Int = {
    if (n <= 0) { 1 }
    else {
        10 * recursionA(n-1)
    }
}

def recursionB(n: Int, m: Int): Int = {
    if (n <= 0) { m }
    else {
        recursionB(n-1, 10 * m)
    }
}

defined function recursionA
defined function recursionB

Let us closely describe how both functions work. Both functions will return if the argument $n \leq 0$. However, if $n > 0$, 
`recursionA` does the following:
- call `recursionA` on $n-1$. 
- Multiply the return value by 10 and return the result.

`recursionB` does the following:
- Call the `recursionB` on $n-1$ and $10m$.
- Pass the return value along _as is_.

The important difference we highlight here is that `recursionA` performs further calculations on the result of the recursive call whereas `recursionB` does not:

~~~
recursionA: wait for recursive call to return. Multiply result by 10 and return it back.

recursionB: wait for recursive call to return and simply return the result back to the caller.
~~~

Therefore, we say `recursionB` is tail recursive since it simply passes along the return value from the recursive calls as is.

However, `recursionA` is not tail recursive since it performs further computing on the return value.

- The factorial function is _not_ tail recursive as is. Why?
- Is the fibonacci function tail recursive?
- Is the function `isPowerOfTwo` tail recursive?
- Is the function `M` tail recursive?

### Tail recursion 

The important reason why we care about tail recursion is that since it simply passes along the return value as is, there is no need to keep its activation record. Simply, when we make the recursive call, we  give this call the current return address so that it can directly pass its result along to the root call. 

<todo draw diagram showing how this can work>
    
The main takeaway is that tail recursive calls can effectively be turned into a while loop  by the compiler, and the stack depth compressed to $1$.

We can convert non-tail recursive recurrences to tail recursive ones.
Here is how we do it for factorial. Note that we added an accumulator
argument `acc` that carries around the product so far.

In [23]:
def tailRecursiveFactorial(n: Int, acc: Int): Int = {
    if (n == 0) { acc }
    else { tailRecursiveFactorial(n-1, acc * n) }
}

def factorial(n: Int): Int = tailRecursiveFactorial(n, 1)

defined function tailRecursiveFactorial
defined function factorial

In [24]:
// Equivalent loop to tail recursive version
def factorialLoop(n: Int): Int = {
    var j = n
    var acc = 1
    while (j >= 0){
        acc = acc * j
        j = j - 1
    }
    return acc
}

defined function factorialLoop

Converting something more complicated like fibonacci to an _equivalent_ tail recursive version is more complicated. We can achive this by using two accumulators and thus effectively translating a while loop version into a tail recursive call.

In [25]:
// j counts up from 1 to n
// acc1 is fibonacci(j)
// acc2 is fibonacci(j-1)
// implement the while loop
// while (j < n) { (acc1, acc2) = (acc1+ acc2, acc1)}
// return acc1

def fibonacci(n: Int, j: Int = 1, acc1: BigInt = 1, acc2: BigInt = 1): BigInt = {
    if (n == 0 ) { BigInt(1) }
    else if (j == n) { acc1 }
    else {
        fibonacci(n, j+1, acc1 + acc2, acc1)
    }
}

defined function fibonacci

In [26]:
val v1 = fibonacci(5)
val v2 = fibonacci(7)

v1: BigInt = 8
v2: BigInt = 21

## Mutually Recursive Functions

Just like we have functions that call themselves, it is possible to have mutually recursive functions that are defined in terms of each other. 
In the example below, m1 calls m2 and m2 calls m1. This is an example of a mutual recursive function. Is it tail recursive? 

In [27]:
def m1 (x: Int): Int = {
    if (x <= 2) { x } 
    else {
        (m2( (x/2+1).toString )).toInt
    } 
}

def m2(s: String): String = {
    if (s.length() <= 1) { s }
    else {
        val t = s.substring(0, s.length() -1 )
        m1(t.toInt).toString
    }
}


defined function m1
defined function m2

In [28]:
val s1 = m1(250)
val s2 = m1(90)
val s3 = m1(1001)
val s4 = m1(30091)

s1: Int = 7
s2: Int = 3
s3: Int = 2
s4: Int = 2

## Recursion and Fun?

See the posted python notebook here. We will translate this to scala soon.
https://github.com/sriram0339/notebooks/blob/master/LSystemsAndFractals.ipynb